In [1]:
from __future__ import division
import pandas as pd
from itertools import combinations_with_replacement
import string


In [2]:

v1 = pd.read_csv("../exp_files/v2_ConceptualMatchingOnly.csv")
v2 = pd.read_csv("../exp_files/v2_MaxMatch.csv")
v3 = pd.read_csv("../exp_files/v2_MaxMismatch.csv")
v4 = pd.read_csv("../exp_files/v2_ValenceMatchingOnly.csv")
v5 = pd.read_csv("../exp_files/v2_SelfPair.csv").

In [5]:
df = pd.concat([v1,v2,v3,v4,v5])

In [9]:
df.head()

,Word1,Word2,FeatureCombo1,FeatureCombo2,FeatureMatch,Group
0,cuddle,rust,physical-positive,physical-negative,ConceptualMatchingOnly,NaN
1,cuddle,slaughter,physical-positive,physical-negative,ConceptualMatchingOnly,NaN
2,cuddle,slash,physical-positive,physical-negative,ConceptualMatchingOnly,NaN
3,cuddle,crash,physical-positive,physical-negative,ConceptualMatchingOnly,NaN
4,cuddle,tear,physical-positive,physical-negative,ConceptualMatchingOnly,NaN


In [37]:
import pandas as pd
import numpy as np

# Set global seed for reproducibility
global_seed = 42
np.random.seed(global_seed)

# Final result list
result_rows = []

# Tracks words used in fallback groups across groups (for cross-group deduplication)
fallback_shared_used_words = set()
fallback_restricted_groups = {'MaxMismatch', 'ValenceMatchingOnly'}

# Group by FeatureMatch
for i, (feature, group) in enumerate(df.groupby('FeatureMatch')):
    group_seed = global_seed + i
    group_shuffled = group.sample(frac=1, random_state=group_seed)

    selected_rows = []
    used_words = set()
    is_self_pair = feature == "SelfPair"
    is_fallback_group = feature in fallback_restricted_groups

    # Primary Selection (no repeats at all within group, unless SelfPair)
    for _, row in group_shuffled.iterrows():
        w1, w2 = row['Word1'], row['Word2']
        if is_self_pair or (w1 not in used_words and w2 not in used_words):
            selected_rows.append(row)
            used_words.update([w1, w2])
        if len(selected_rows) == 10:
            break

    # If <10 selected and not SelfPair — enter fallback
    if len(selected_rows) < 10 and not is_self_pair:
        print(f"⚠️ Only {len(selected_rows)} valid rows for FeatureMatch '{feature}' — using fallback sampling")

        fallback_group = group.sample(frac=1, random_state=group_seed + 1000)
        fallback_selected = []
        used_word1 = set()
        used_word2 = set()

        # Phase 1: Strict (no repeats within or across fallback groups)
        for _, row in fallback_group.iterrows():
            if len(fallback_selected) == 10:
                break
            w1, w2 = row['Word1'], row['Word2']
            if (w1 in used_word1 or w2 in used_word2 or
                (is_fallback_group and (w1 in fallback_shared_used_words or w2 in fallback_shared_used_words))):
                continue
            fallback_selected.append(row)
            used_word1.add(w1)
            used_word2.add(w2)
            if is_fallback_group:
                fallback_shared_used_words.update([w1, w2])

        # Phase 2: Relax cross-group restriction
        if len(fallback_selected) < 10:
            print(f"ℹ️ Relaxing cross-group uniqueness for '{feature}'")
            for _, row in fallback_group.iterrows():
                if len(fallback_selected) == 10:
                    break
                w1, w2 = row['Word1'], row['Word2']
                if w1 in used_word1 or w2 in used_word2:
                    continue
                fallback_selected.append(row)
                used_word1.add(w1)
                used_word2.add(w2)

        # Phase 3: Allow Word1 repeats
        if len(fallback_selected) < 10:
            print(f"ℹ️ Allowing Word1 repeats for '{feature}' to reach 10")
            for _, row in fallback_group.iterrows():
                if len(fallback_selected) == 10:
                    break
                w1, w2 = row['Word1'], row['Word2']
                if w2 in used_word2:
                    continue
                fallback_selected.append(row)
                used_word2.add(w2)

        print(f"✅ Fallback: Selected {len(fallback_selected)} rows for FeatureMatch '{feature}'")
        result_rows.extend(fallback_selected)
    else:
        result_rows.extend(selected_rows)

# Final DataFrame
final_df = pd.DataFrame(result_rows)
print(f"\n✅ Final total rows: {len(final_df)}")



✅ Final total rows: 50


In [38]:
len(final_df)

50

In [39]:
final_df.groupby(["FeatureMatch"])["FeatureMatch"].count()

FeatureMatch
ConceptualMatchingOnly    10
MaxMatch                  10
MaxMismatch               10
SelfPair                  10
ValenceMatchingOnly       10
Name: FeatureMatch, dtype: int64

In [40]:
final_df = final_df.drop(columns=['Group'])

In [42]:
final_df.to_csv("../exp_files/pilot.csv")